In [10]:
import numpy as np
# import units as u
import numpy as np
import pandas as pd



import rasterio
from rasterio.plot import show_hist, show
import geopandas as gpd
import rasterstats

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt

from icecream import ic

In [2]:
# shape file definining administrative borders 
states_path = "data/states/nga_admbnda_adm1_osgof_20161215.shp"
states = gpd.read_file(states_path)

In [6]:
# raster data sources => wind, temperature, solar flux 

data_paths = {
    "wind_speed": "data/Nigeria_MeanWindSpeed/NGA_wind-speed_100m.tif",
    "temperature": "data/Nigeria_AvgDailyTotals_GlobalSolarAtlas_GEOTIFF/TEMP.tif",
    "solar_flux": "data/Nigeria_AvgDailyTotals_GlobalSolarAtlas_GEOTIFF/DNI.tif",
}

In [8]:
def get_state_means(data_path, states=states):
    data = rasterio.open(data_path)
    data_array = data.read()[0]
    affine = data.transform

    # calculate zonal statistics
    averaged_data = rasterstats.zonal_stats(states, data_array, affine=affine, stats = ["mean"], geojson_out=True)

    state_wind_average = []
    for item in averaged_data:
        state_wind_average.append((item["properties"]["admin1Name"],
        item["properties"]["mean"]))

    df = pd.DataFrame(state_wind_average, columns=["state", "value"])
    return df


In [11]:
data_averages = {}
for name, path in data_paths.items():
    data_averages[name] = get_state_means(path, states=states)

In [19]:
avg_vals = {n: i["value"] for n, i in data_averages.items()}

In [13]:
def calculate_power_panel(F_cur, T_a, w):
    """
    F_cur = _ # W/m^2 current solar flux normal to the panel 
    T_a = _ # K / ºC, air temperature the panel is exposed to 
    w = _ # m/s, wind speed panel is exposed to 

    => all of these are 2x2 arrays with spatial information 
    
    """

    # values that change based on design! 
    D_f = 0.864 # derating factor, product of correction factors for additional processes affecting solar output, Table 5.2 
    E_panel = 0.18 # solar panel efficiency obtained under standard test conditions -- Ex 5.2
    A_panel = 1.5 # m^2, surface area of the pane -- Ex 5.2
    
    # 5.9, cell temperature, empirical so units do not eqate 
    T_c = T_a + 0.32 * (F_cur/(8.91 + 2*w))

    b_ref = 0.0025 # / K, temperature coefficient 
    T_th = 55 # K, threshold temeprature 
    T_ref = 298.15 # K, reference temperature 
    # 5.8, correction for cell temperature 
    C_temp = 1 - b_ref * np.maximum( np.minimum(T_c - T_ref, T_th ), 0 )

    # 5.7 actual AC power output from a solar panel at a given time 
    # P_ac = p_mpp_stc * C_temp * D_f / F_1000
    P_ac = F_cur * A_panel * E_panel * C_temp * D_f # units?? # should i be considering the rating of the panel as well? 

    return P_ac


In [22]:
state_power_potential  = calculate_power_panel(F_cur=avg_vals["solar_flux"], T_a=avg_vals["temperature"], w=avg_vals["wind_speed"])